# XMM-LSS master catalogue
## Preparation of HST CANDELS-UDS data

The UDS survey appears to be a cross match between CANDELS-3D-HST with various other multiwavelength catalogues that we are already including across XMM-LSS. I therefore choose not to use it here.